In [8]:
import os
import numpy as np
import torch
import matplotlib.pyplot as plt

In [9]:
# EMBED_DIR = "/home/doyeolkim/vox_emb/test/"
EMBED_DIR = "/home/doyeolkim/vox_emb/train/"
# EMBED_DIR = "/home/doyeolkim/libri_emb/valid2/"

In [10]:
def get_spk_mat(target_path):
    
    '''
    target_path 내에 있는 모든 임베딩 벡터 npy 파일을
    평균을 내서 저장.
    '''
    
    np_list = []
    
    for root, directories, filenames in os.walk(target_path):
        for filename in filenames:
            if str.endswith(filename, '.npy'):
                emb = np.load(root + '/' + filename)
                assert emb.shape == (10,512)                
                emb = np.mean(emb, axis=0)
                emb = np.array(torch.nn.functional.normalize(torch.tensor(emb), p=2, dim=0))
                np_list.append(emb)
    
    mat = np.array(np_list)
    return mat

In [11]:
spk_list = []

for i in os.listdir(EMBED_DIR):
    if os.path.isdir(EMBED_DIR+i):
        spk_list.append(i)
        
LEAST_FILE_NUM = 3
total_file_num = 0

spk_cat_num = {}
spk_file_num = {}

for i, spk in enumerate(spk_list):
    print('\rprocessing ({}/{})'.format(i+1, len(spk_list)), end='')
    for file in os.listdir(EMBED_DIR+spk):
        cat_path = os.path.join(EMBED_DIR+spk, file)
        if os.path.isdir(cat_path):
            file_num = len(os.listdir(cat_path))
            if file_num < LEAST_FILE_NUM:
                continue
            total_file_num += file_num
            spk_file_num[spk] = spk_file_num.get(spk, 0) + file_num
            spk_cat_num[spk] = spk_cat_num.get(spk, 0) + 1

print()
print(total_file_num)

spk_count = 0
for spk in spk_list:
    if spk_cat_num.get(spk, 0) == 1: continue
    spk_count += 1

print(spk_count)

processing (5994/5994)
1024580
5979


In [ ]:
spk_list = []
mean_list = []
var_list = []
spk_mat_list = []

for i in os.listdir(EMBED_DIR):
    if os.path.isdir(EMBED_DIR+i):
        spk_list.append(i)


for i, spk in enumerate(spk_list):
    if i == 100: break
    spk_mat_list.append(get_spk_mat(EMBED_DIR+spk))
    print('\rprocessing ({}/{})'.format(i+1, len(spk_list)), end='')
print()


for i, spk in enumerate(spk_list):
    if i == 100: break
    spk_mat = spk_mat_list[i]
    comat = np.dot(spk_mat, spk_mat.T)
    comat = 1 - comat
    distance_arr = comat[np.tril_indices(n=comat.shape[0], k=-1)]
    mean_list.append(np.mean(distance_arr))
    var_list.append(np.var(distance_arr))
    print('\rprocessing ({}/{})'.format(i+1, len(spk_list)), end='')
print()


# fig, ax = plt.subplots()
# ax2 = ax.twinx()
# ax.plot(x[0:len(mean_list)], mean_list, 'b-')
# ax2.plot(x[0:len(var_list)], var_list, 'r-')
# plt.show()

plt.scatter(mean_list, var_list)
plt.show()

In [ ]:
# MIN = np.inf
# MIN_IDX = 0
# for i in range(0, len(mean_list)):
#     if mean_list[i]*var_list[i] < MIN:
#         MIN_IDX = i
#         MIN = mean_list[i]*var_list[i]
        
# print(MIN_IDX, mean_list[MIN_IDX], var_list[MIN_IDX])

In [ ]:
# print(spk_list[1926])

In [ ]:
cate_list = []
cat_mat_list = []
cat_mean_list = []
cat_var_list = []


for i, spk in enumerate(spk_list):
    if i == 100: break
    for file in os.listdir(EMBED_DIR+spk):
        if os.path.isdir(os.path.join(EMBED_DIR+spk, file)):
            cate_list.append(file)
            cat_mat_list.append(get_spk_mat(os.path.join(EMBED_DIR+spk, file)))
    
    print('\rprocessing ({}/{})'.format(i+1, len(spk_list)), end='')


print()
for i, cat in enumerate(cate_list):
    cat_mat = cat_mat_list[i]
    if cat_mat.shape[0] < 2: continue
    comat = np.dot(cat_mat, cat_mat.T)
    comat = 1 - comat
    distance_arr = comat[np.tril_indices(n=comat.shape[0], k=-1)]
    cat_mean_list.append(np.mean(distance_arr))
    cat_var_list.append(np.var(distance_arr))
    print('\rprocessing ({}/{})'.format(i+1, len(cate_list)), end='')

In [ ]:
plt.scatter(cat_mean_list, cat_var_list)
plt.show()

In [ ]:
print(np.min(np.array(cat_mean_list)), np.min(np.array(cat_var_list)))